this notebook will be mainly used for the capstone project.

In [1]:
#part 1

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head(10)
#ignore borough not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#use lambda function to join same postcodes
df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
#borough exists and nei. not exists = set nei. == bor.
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']
#notebook clean
#my work: following assignment instructions
#assumptions: no errors in df
print ("no. rows= ",df.shape[0])

#part 2

#lat/lon cols
geo_coords=pd.read_csv("https://cocl.us/Geospatial_data")
df['Latitude']=geo_coords['Latitude'].values
df['Longitude']=geo_coords['Longitude'].values
df



no. rows=  103


,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Oakridge,Golden Mile",43.711112,-79.284577
M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest",43.716316,-79.239476


In [8]:
#part 3

'''
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs
that contain the word Toronto
and then replicate the same analysis
we did to the New York City data.
- to generate maps to visualize your neighborhoods and how they cluster together. 
'''


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [18]:
#new df only borough, toronto

toro_df = df[ df.Borough.str.contains('Toronto') ]
toro_df.reset_index(inplace=True)
toro_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Rathnelly,Deer Park,South Hill,Summerhill West...",43.686412,-79.400049


In [19]:
#working with Toronto df
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
# create map of Toronto using latitude and longitude values
map_toro = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toro_df['Latitude'], toro_df['Longitude'], toro_df['Borough'], toro_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7).add_to(map_toro)
    
map_toro

In [21]:
#import libs for json and fioursquare mapping function
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

In [151]:
# The code was removed by Watson Studio for sharing.

Your credentails:CLIENT_ID and CLIENT_SECRET have been hidden


In [103]:
#chosen University of Toronto,Harbord neibourhood (index[25]) to explore, set up attributes

neighborhood_latitude = toro_df.loc[25, 'Latitude']
neighborhood_longitude = toro_df.loc[25, 'Longitude']
neighborhood_name = toro_df.loc[25, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of University of Toronto,Harbord are 43.6626956, -79.4000493.


In [104]:
#use get function, get top 30 venues within radius= 100 of the beaches
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display GET request URL

'https://api.foursquare.com/v2/venues/explore?&client_id=YTA22TRQRIY2KC2TXWVL2BBOG31XW41G5EETLCRYOIXRBDLJ&client_secret=YPISXA1SS1EQRROIRSNRLAOL1PL3FEHNKC1G1V42TNDRR5PH&v=20180604&ll=43.6626956,-79.4000493&radius=500&limit=100'

In [105]:
#read url via json
results = requests.get(url).json()
results
#single result,=> higher radius and limit

{'meta': {'code': 200, 'requestId': '5d7033bfa4b51b002c2053e1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 43.6671956045, 'lng': -79.39384042790832},
   'sw': {'lat': 43.6581955955, 'lng': -79.4062581720917}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5362c366498e602fbe1db395',
       'name': 'Yasu',
       'location': {'address': '81 Harbord St.',
        'lat': 43.66283719650635,
        'lng': -79.40321739973975,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.662

In [106]:
#convert url json to df
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [110]:


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Piano Piano,Italian Restaurant,43.662949,-79.402898
2,Rasa,Restaurant,43.662757,-79.403988
3,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
4,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703


In [111]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

35 venues were returned by Foursquare.


In [115]:
#generate map to visualize venues within university neighborhood
uni_ven_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=17)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(uni_ven_map) 
    
uni_ven_map

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [122]:


### Get venues for each neighborhood using above function
tor_bor_venues = getNearbyVenues(names=toro_df['Neighbourhood'],
                                   latitudes=toro_df['Latitude'],
                                   longitudes=toro_df['Longitude']
                                  )



The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Rathnelly,Deer Park,South Hill,Summerhill West,Forest Hill SE
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Harbourfront East,Union Station,Toronto Islands
Toronto Dominion Centre,Design Exchange
Victoria Hotel,Commerce Court
Roselawn
Forest Hill West,Forest Hill North
Yorkville,North Midtown,The Annex
University of Toronto,Harbord
Kensington Market,Chinatown,Grange Park
King and Spadina,South Niagara,Island airport,Railway Lands,CN Tower,Bathurst Quay,Harbourfront West
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Parkdale Village,Exhibition Place,Brockton
The Junction South,High Park
Roncesvalles,Parkda

In [130]:
#df with duplicate neighbourhoods => groupby and count function from labs
tor_bor_venues.head()
tor_bor_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
Central Bay Street,85,85,85,85,85,85
Christie,15,15,15,15,15,15


In [134]:
print('There are {} unique categories.'.format(len(tor_bor_venues['Venue Category'].unique())))

There are 234 unique categories.


In [137]:
#one hot encoding from labs, used to convert categorical to numerical values
# one hot encoding
toronto_onehot = pd.get_dummies(tor_bor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = tor_bor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
#groupby and mean value
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,Richmond,King",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.010000,0.000000,0.000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000
2,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.011765,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011765,...,0.000000,0.000000,0.000,0.000000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000
4,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.011494,0.011494,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011494,...,0.011494,0.011494,0.000,0.000000,0.00,0.000000,0.011494,0.011494,0.000000,0.011494
6,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.025,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Dufferin,Dovercourt Village",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Forest Hill West,Forest Hill North",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [140]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Adelaide,Richmond,King----
          venue  freq
0   Coffee Shop  0.08
1          Café  0.05
2           Bar  0.04
3    Steakhouse  0.04
4  Burger Joint  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Fast Food Restaurant  0.07
1  Gym / Fitness Center  0.07
2    Light Rail Station  0.07
3               Brewery  0.07
4            Restaurant  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.06
2      Sandwich Place  0.05
3                Café  0.05
4      Ice Cream Shop  0.04


----Christie----
           venue  freq
0  Grocery Store  0.20
1           Café  0.20
2           Park  0.13
3     Baby Store  0.07
4     Restaurant  0.07


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07


In [141]:
'''
Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.
'''
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [142]:


num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,Bar,Steakhouse,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Farmers Market
2,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Garden,Comic Shop,Gym / Fitness Center,Recording Studio
3,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint
4,Christie,Grocery Store,Café,Park,Italian Restaurant,Nightclub
5,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant
6,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Café,Italian Restaurant
7,Davisville North,Pizza Place,Breakfast Spot,Sandwich Place,Food & Drink Shop,Clothing Store
8,"Dufferin,Dovercourt Village",Supermarket,Pharmacy,Bakery,Music Venue,Park
9,"Forest Hill West,Forest Hill North",Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint


In [143]:
'''
4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.
'''
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]



array([0, 0, 0, 0, 0], dtype=int32)

In [146]:
toronto_merged = toro_df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Diner,Fast Food Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pet Store,Park,Brewery,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Wings Joint,Dog Run


In [149]:
#from lab
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
#END OF WK 3